In [ ]:
%run ./../data/_meta.ipynb

In [ ]:
import pandas as pd
from pathlib import Path
from gensim.corpora import Dictionary

In [ ]:
PROC_DATASET_PATH = f'{PROC_DATA_DIR}/proc_dataset.pkl'
DICTIONARY_PATH = f'{PROC_DATA_DIR}/dictionary.dict'

In [ ]:
if not Path(PROC_DATASET_PATH).is_file() or not Path(DICTIONARY_PATH).is_file():
    if not Path(DATASET_PATH).is_file():
        %run ./../data/_generate-dataset.ipynb
    %run ./../data/_preprocess-dataset.ipynb
else:
    df, dictionary = pd.read_pickle(PROC_DATASET_PATH), Dictionary.load(DICTIONARY_PATH)